This is a sanity check/Python prototype for PageRank for dense matrices, that's implemented in CUDA C/C++ (i.e. compare what we compute here against simple examples with results from the CUDA C/C++ implementation, files `main_pagerank.cu`, `pagerank.cu` (the "meat" of the PageRank algorithm), `pagerank.h` (CUDA C/C++ header file)

In [1]:
import sympy

In [2]:
import numpy as np

In [3]:
import scipy

In [4]:
from scipy.sparse import csr_matrix

In [5]:
intptr = np.array([0, 1, 2, 3, 4, 8, 9, 10, 12, 12])
J = np.array([4, 6, 7, 1, 0, 1, 2, 3, 6, 1, 5, 8]) # col
data = np.array([1.000000, 0.500000, 1.000000, 0.333333, 1.000000, 0.333333, 1.000000, 1.000000, 0.500000, 0.333333, 1.000000, 1.000000])

In [6]:
mymtx=csr_matrix((data,J,intptr))

In [7]:
mymtx.toarray()

array([[ 0.      ,  0.      ,  0.      ,  0.      ,  1.      ,  0.      ,
         0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.5     ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  1.      ,  0.      ],
       [ 0.      ,  0.333333,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ],
       [ 1.      ,  0.333333,  1.      ,  1.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.5     ,  0.      ,  0.      ],
       [ 0.      ,  0.333333,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  1.      ,
         0.      ,  0.      ,  1.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      

In [8]:
mymtx.toarray().shape

(9, 9)

In [9]:
for row in mymtx.toarray():
    print row

[ 0.  0.  0.  0.  1.  0.  0.  0.  0.]
[ 0.   0.   0.   0.   0.   0.   0.5  0.   0. ]
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.]
[ 0.        0.333333  0.        0.        0.        0.        0.        0.
  0.      ]
[ 1.        0.333333  1.        1.        0.        0.        0.        0.
  0.      ]
[ 0.   0.   0.   0.   0.   0.   0.5  0.   0. ]
[ 0.        0.333333  0.        0.        0.        0.        0.        0.
  0.      ]
[ 0.  0.  0.  0.  0.  1.  0.  0.  1.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [10]:
for column_index in range(9):
    print sum( mymtx.toarray()[i][column_index] for i in range(9) )
        

1.0
0.999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [34]:
N = 9
# prepare x
# initiate x vector with 1/N, y with 0 
x = np.ones(N) / N
E = np.ones(N) / N
y = np.zeros(N)

In [29]:
np.vstack(x)

array([[ 0.11111111],
       [ 0.11111111],
       [ 0.11111111],
       [ 0.11111111],
       [ 0.11111111],
       [ 0.11111111],
       [ 0.11111111],
       [ 0.11111111],
       [ 0.11111111]])

In [22]:
y

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [30]:
np.matmul(mymtx.toarray(),np.vstack(x))

array([[ 0.11111111],
       [ 0.05555556],
       [ 0.11111111],
       [ 0.037037  ],
       [ 0.37037033],
       [ 0.05555556],
       [ 0.037037  ],
       [ 0.22222222],
       [ 0.        ]])

In [31]:
alpha=0.85

In [35]:
alpha*np.matmul(mymtx.toarray(),np.vstack(x)) + (1.-alpha)*np.vstack(E)

array([[ 0.11111111],
       [ 0.06388889],
       [ 0.11111111],
       [ 0.04814812],
       [ 0.33148145],
       [ 0.06388889],
       [ 0.04814812],
       [ 0.20555556],
       [ 0.01666667]])

In [36]:
newx = alpha*np.matmul(mymtx.toarray(),np.vstack(x)) + (1.-alpha)*np.vstack(E)

In [43]:
newx-np.vstack(x)

array([[ 0.        ],
       [-0.04722222],
       [ 0.        ],
       [-0.06296299],
       [ 0.22037034],
       [-0.04722222],
       [-0.06296299],
       [ 0.09444444],
       [-0.09444444]])

In [44]:
np.sum( np.abs(newx-np.vstack(x)) )

0.62962966111111107

In [45]:
np.sum( newx)

0.99999990555555551

In [50]:
# defining a function,
def pagerank(alpha,A,x,E):
    newx = alpha*np.matmul(A,np.vstack(x)) + (1.-alpha)*np.vstack(E)
    err = np.sum( np.abs(newx-np.vstack(x)) )
    print "Error : ", err
    summation = np.sum( newx)
    print "Sum   : ", summation
    print newx
    return newx

In [48]:
newx=pagerank(alpha,mymtx.toarray(),newx,E

Error :  0.535185171806
Sum   :  0.999999865417


In [49]:
newx

array([[ 0.2984259 ],
       [ 0.03712962],
       [ 0.19138889],
       [ 0.0347685 ],
       [ 0.26458329],
       [ 0.03712962],
       [ 0.0347685 ],
       [ 0.08513889],
       [ 0.01666667]])

In [51]:
newx=pagerank(alpha,mymtx.toarray(),newx,E)

Error :  0.416998452999
Sum   :  0.999999854044
[[ 0.24156246]
 [ 0.03144328]
 [ 0.08903472]
 [ 0.02718671]
 [ 0.47308251]
 [ 0.03144328]
 [ 0.02718671]
 [ 0.06239351]
 [ 0.01666667]]


In [52]:
newx=pagerank(alpha,mymtx.toarray(),newx,E)

Error :  0.354448680216
Sum   :  0.999999849211
[[ 0.4187868 ]
 [ 0.02822102]
 [ 0.06970115]
 [ 0.02557559]
 [ 0.3296919 ]
 [ 0.02822102]
 [ 0.02557559]
 [ 0.05756012]
 [ 0.01666667]]
